In [2]:
!pip install moviepy
!pip install imageio==2.4.1
!pip install facenet_pytorch

  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110744 sha256=f6f2390dde890274a7ec6a9df508a3c467059c50b52f9f74fbae0903d5079569
  Stored in directory: c:\users\khyun\appdata\local\pip\cache\wheels\29\15\e4\4f790bec6acd51a00b67e8ee1394f0bc6e0135c315f8ff399a
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.0
    Uninstalling decorator-5.1.0:
      Successfully uninstalled decorator-5.1.0
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=8105bf4db9085c1b0cb936df93cd4942bba1c1205c9470ef25f889de5fbde5e5
  Stored in directory: c:\users\khyun\appdata\local\pip\cache\wheels\b7\44\b7\2e7cc9c5fe4a893b9cc83a010d4410557bedf6cf3b5829f497
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:


ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'c:\\anaconda3\\scripts\\imageio_download_bin.exe'
Consider using the `--user` option or check the permissions.



In [3]:
!pip install dlib

  Running setup.py clean for dlib
Failed to build dlib
    Running setup.py install for dlib: started
    Running setup.py install for dlib: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\khyun\\AppData\\Local\\Temp\\pip-install-c3jzw58k\\dlib_2d97340d68ee4c46a337bcf156a90e07\\setup.py'"'"'; __file__='"'"'C:\\Users\\khyun\\AppData\\Local\\Temp\\pip-install-c3jzw58k\\dlib_2d97340d68ee4c46a337bcf156a90e07\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\khyun\AppData\Local\Temp\pip-wheel-0beff0uk'
       cwd: C:\Users\khyun\AppData\Local\Temp\pip-install-c3jzw58k\dlib_2d97340d68ee4c46a337bcf156a90e07\
  Complete output (8 lines):
  running bdist_wheel
  running build
  running build_py
  package init file 'tools\python\dlib\__init__.py' not found (or not a regular

In [4]:
import os
import random
import datetime
import argparse
import time
import numpy as np
from torchvision import models
import torch.nn as nn
import torch
import dlib
import imutils
from imutils.video import VideoStream
from imutils import face_utils
import cv2
# from moviepy.editor import *
from moviepy.editor import VideoFileClip, concatenate_videoclips

ModuleNotFoundError: No module named 'dlib'

In [4]:
%cd /content/drive/MyDrive/Deep-Project/
!cp -r cross_cut /content/
# !cp cross_cut/blackpink_7.mp4 /content/cross_cut/

# %cd /content/cross_cut
# !mv shape_predictor_68_face_landmarks.dat /content/

/content/drive/MyDrive/Deep-Project


In [5]:
class Crosscut:
    def __init__(self, dist_obj, video_path, output_path):
        self.videos_path = video_path                       # 사용할 비디오 저장경로
        self.output_path = output_path                      # 결과물 저장경로
        self.min_time = 1000.0                              # 비디오중 가장 짧은 길이 저장 변수, 초기화값
        video_num = len(os.listdir(self.videos_path))       # 비디오 갯수
        self.start_times = [0] * video_num                  # 비디오별 시작시간
        self.window_time = 5                               # 비디오별 비교 시간 범위 (start +  window)  
        self.padded_time = 10                               # 영상 전환전 최소 유지시간
        self.dist_obj = dist_obj                            # 영산간 거리값
        self.audioclip = None                               # 음성 데이터
        self.extracted_clips_array = []                     # 시작점이 정렬된 비디오
        self.con_clips = []                                 # 결과물 저장
    
    def video_alignment(self):      # 영상들을 extracted_clips_array에 저장
        for i in range(len(os.listdir(self.videos_path))):
            video_path = os.path.join(self.videos_path, sorted(os.listdir(self.videos_path))[i])
            if '.ipynb_checkpoints' in video_path :
                continue
            clip = VideoFileClip(video_path)
            clip = clip.subclip(self.start_times[i], clip.duration)

            if self.min_time > clip.duration:               # 가장 짧은 비디오길이로 self.audio와 self.min_time 업데이트
                self.audioclip = clip.audio
                self.min_time = clip.duration
            self.extracted_clips_array.append(clip)
        print(f'LOGGER -- {len(self.extracted_clips_array)} will be mixed')
    
    def select_next_clip(self, t, current_idx):
        cur_t = t
        next_t = min(t + self.window_time, self.min_time)
        reference_clip = self.extracted_clips_array[current_idx].subclip(cur_t, next_t)     # 기준 영상 설정
        d = float('Inf')
        cur_clip = None
        min_idx = (current_idx + 1) % len(self.extracted_clips_array)                       # 다음 영상 초기화

        for video_idx in range(len(self.extracted_clips_array)):                            # 전체 영상 비교
            if video_idx == current_idx:
                continue
            clip = self.extracted_clips_array[video_idx].subclip(cur_t, next_t)
            cur_d, plus_frame = self.dist_obj.distance(reference_clip, clip)                 # reference와 clip 간 거리 비교
            print(f'current video_idx : {current_idx} , compare video_idx : {video_idx}, distance : {cur_d}, compared end_time : {cur_t + plus_frame}')
            if d > cur_d:
                d = cur_d
                min_idx = video_idx
                next_t = cur_t + plus_frame
                cur_clip = reference_clip.subclip(0, plus_frame)                     # plus_frame 까지는 refernce, 이후로는 min_idx 영상 사용
        
        if cur_clip : 
            clip = cur_clip
        else :
            clip = reference_clip
        print(f'insert video idx : {min_idx}, insert time : {t} to {next_t}')
        self.con_clips.append(clip)

        t = next_t
        return t, min_idx

    def add_padding(self, t, next_idx):                 # con_clips에 next_idx 영상의 t시간 추가
        print(f'insert idx : {next_idx}')
        pad_clip = self.extracted_clips_array[next_idx].subclip(t, min(self.min_time, t + self.padded_time))
        self.con_clips.append(pad_clip)

        t = min(self.min_time, t + self.padded_time)
        return t, next_idx

    def write_video(self):
        final_clip = concatenate_videoclips(self.con_clips)
        if self.audioclip != None:
            print('Not None')
            final_clip.audio = self.audioclip
        final_clip.write_videofile(self.output_path, threads=5)        
        return final_clip
    
    def generate_video(self):                   # video 생성 main 함수
        self.video_alignment()
        t = 10
        current_idx = 0
        self.con_clips.append(self.extracted_clips_array[current_idx].subclip(0, min(t, int(self.min_time))))

        while t < int(self.min_time/10):           # min_time(audio 길이) 까지 반복
            t, min_idx = self.select_next_clip(t, current_idx)
            t, current_idx = self.add_padding(t, min_idx)
        
        final_clip = self.write_video()
        return final_clip

In [6]:
class RandomDistance():
    def distance(self, reference_clip, compare_clip):
        dur_end = min(reference_clip.duration, compare_clip.duration)
        return random.randrange(1,100), min(dur_end, random.randrange(50,60))

In [7]:
class FaceDistance:         # facedistance 계산
    def __init__(self, shape_predictor_path, face_embedding_penalty = None):
        self.skip_frame_rate = 4                                    # 4frame 마다 distance 계산
        self.minimax_frames = 5                                     # 연속되 5frame을 계산
        self.shape_predictor = shape_predictor_path
        self.face_embedding_penalty =face_embedding_penalty

    def extracted_landmark(self, reference_clip, compare_clip):
        self.clips = [reference_clip, compare_clip]                 # 영상 저장
        detector = dlib.get_frontal_face_detector()                 # face position detector
        predictor = dlib.shape_predictor(self.shape_predictor)      # landmark detector
        clips_frame_info = []
        
        for clip in self.clips:
            i = 0
            every_frame_info = []
            while True:
                frame = clip.get_frame(i * 1.0 / clip.fps)          # clip에서 frame 을 image로 추출
                i += self.skip_frame_rate                           # 다음 i-th frame 
                if (i * 1.0 / clip.fps) > clip.duration:
                    break
                
                frame = imutils.resize(frame, width = 800)          # 이미지 축소로 연산속도 향상
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)      # gray 채널만 사용해서 연산속도 향상
                rects = detector(gray, 0)                           # face position box 추출

                if len(rects)> 0:
                    max_width = 0
                    max_rect = None
                    for rect in rects:
                        if int(rects[0].width()) > max_width:       # 가장 큰 얼굴만 사용
                            max_rect = rect
                    shape = predictor(gray, max_rect)
                    shape = imutils.face_utils.shape_to_np(shape)
                    every_frame_info.append(shape)
                
                else :                                              # face가 없으면 빈 list 전달
                    every_frame_info.append([])
            
            clips_frame_info.append(np.array(every_frame_info))
        cv2.destroyAllWindows()
        return clips_frame_info
    
    def embedding_cosine_distance(self, reference_frame, compare_frame):
        face_detector = MTCNN(select_largest = True)                        # face detector
        embed_model = InceptionResnetV1(pretrained = 'vggface2').eval()     # face embedding model

        reference_frame = np.array(reference_frame)
        compare_frame = np.array(compare_frame)
        try :                                                               # face_detector return값이 없을경우 오류가 나기에 try사용
            reference_frame_detected = face_detector(reference_frame)
            compare_frame_detected = face_detector(compare_frame)
        
        except :
            cosine_dist=1
            return cosine_dist

        reference_frame_embed = embed_model(reference_frame_detected.unsqueeze(0)).detach().numpy()   
        compare_frame_embed = embed_model(compare_frame_detected.unsqueeze(0)).detach().numpy()
        reference_frame_embed = np.squeeze(reference_frame_embed)
        compare_frame_embed = np.squeeze(compare_frame_embed)

        cosine_dist = 1 - np.dot(reference_frame_embed, compare_frame_embed) /  \
                     (np.linalg.norm(reference_frame_embed) * np.linalg.norm(compare_frame_embed))
        
        return cosine_dist

    def get_all_frame_distance(self, clips_frame_info, min_size):
        dist_arr = []
        for i in range(min_size -1):
            if len(clips_frame_info[0][i]) > 0 and len(clips_frame_info[1][i+1]) > 0 :
                l = 36      # left eye landmark
                r = 45      # right eye landmark
                #clips_frame_info[0][i][l][0] : 0: 첫번째 영상 // i : i번째 landmark 정보 // l : lefteye // 0 : x좌표
                left_eye = ((clips_frame_info[0][i][l][0] - clips_frame_info[1][i+1][l][0]) **2 + \
                            (clips_frame_info[0][i][l][1] - clips_frame_info[1][i+1][l][1]) **2) **0.5      # frame간 left eye 거리 
                right_eye = ((clips_frame_info[0][i][r][0] - clips_frame_info[1][i+1][r][0]) **2 + \
                            (clips_frame_info[0][i][r][1] - clips_frame_info[1][i+1][r][1]) **2) **0.5      # frame간 right eye 거리
                total_diff = left_eye + right_eye
                dist_arr.append(total_diff)
            else : 
                dist_arr.append(None)
        return dist_arr
    
    def distance(self, reference_clip, compare_clip):
        clips_frame_info = self.extracted_landmark(reference_clip, compare_clip)      # landmark 추출
        min_size = min(len(clips_frame_info[0]), len(clips_frame_info[1]))          # 1, 2번째 영상중 짧은영상을 기준 
        dist_arr = self.get_all_frame_distance(clips_frame_info, min_size)          # landmark distance 계산
        clips = [reference_clip, compare_clip]
        minimax_frames = self.minimax_frames
        min_diff = np.float('Inf')
        min_idx = 0

        for i in range(min_size - (minimax_frames -1)):
            start_minmax_idx = 0 if (i - minimax_frames) < 0 else i - minimax_frames
            if (None not in dist_arr[start_minmax_idx : i + minimax_frames]):       # 얼굴이 없는경우 None
                tmp_max = np.max(dist_arr[start_minmax_idx : i + minimax_frames])   # dist_arr의 max를 대표값으로 설정
                if min_diff > tmp_max:
                    min_diff = tmp_max
                    min_idx = i
        
        if self.face_embedding_penalty != None and min_diff < np.float("Inf"):
            ref_frame = reference_clip.get_frame(min_idx * 1.0 / reference_clip.fps)
            frame = compare_clip.get_frame(min_idx * 1.0 / compare_clip.fps)
            cosine_dist = self.embedding_cosine_distance(ref_frame, frame)
            min_diff += cosine_dist * self.face_embedding_penalty
        
        return min_diff, (min_idx * self.skip_frame_rate) / self.clips[0].fps

In [8]:
class PoseDistance:
    def __init__(self):
        self.SKIP_FRAME_RATE = 10
        self.MINIMAX_FRAME = 4
        self.model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        self.model.eval()
        os.environ["KMP_DUPLICATE_LIB_OK"] = 'True'
    
    def extract_boxes(self, reference_clip, compare_clip):
        self.clips = [reference_clip, compare_clip]
        clips_frame_info = []
        for clip in self.clips:
            i = 0
            every_frame_info = []
            while True :
                i += self.SKIP_FRAME_RATE
                if (i * 1.0 / clip.fps) > clip.duration:
                    break

                # model에 넣을 input transform    
                frame = clip.get_frame(i * 1.0 / clip.fps)
                frame = imutils.resize(frame, width = 480)  # 이미지 축소로 연산속도 향상
                frame = frame / 255                         # model의 input range(0,1)
                frame = np.transpose(frame, (2,0,1))        # get_frame의 output (H, W, C) -> model의 input (C, H, W)으로 변경

                if torch.cuda.is_available():               #GPU 사용
                    x = [torch.from_numpy(frame).float().cuda()]       # model에 넣기전 tensor, cuda로 변환
                    self.model.to('cuda')
                else :
                    x = [torch.from_numpy(frame).float()]       # model에 넣기전 tensor로 변환

                predictions = self.model(x)
                prediction = predictions[0]
                # faster-RCNN output을 변형
                each_box_list = zip(prediction['boxes'].tolist(), prediction['labels'].tolist(), prediction['scores'].tolist())
                # 사람(1), 0.95% 이상 필터
                filtered_box_list = filter(lambda x : x[1] == 1 and x[2] >=  0.95, each_box_list)
                # center 값 저장
                filtered_center_dot_list = list(map(lambda x : [(x[0][0] + x[0][2]) / 2, (x[0][1] + x[0][3]) /2], filtered_box_list))
                sorted_dot_list = sorted(filtered_center_dot_list, key = lambda x : x[0])
                every_frame_info.append(sorted_dot_list)
            clips_frame_info.append(np.array(every_frame_info))
        return clips_frame_info
    
    def get_all_frame_distance(self, clips_frame_info, min_size):
        dist_arr = list()
        for i in range(min_size):
            if len(clips_frame_info[0][i]) > 0 and len(clips_frame_info[1][i]) > 0:
                ref_frame_dots = clips_frame_info[0][i]
                compare_frame_dots = clips_frame_info[1][i]
                min_dot_num = min(len(ref_frame_dots), len(compare_frame_dots))             # 두 영상 모두 dot이 있어야 하므로 최소 개수 설정
                dot_num_diff = abs(len(ref_frame_dots) - len(compare_frame_dots))           # dot 개수 차이 저장
                # 영상의 점이 없을경우 penalty 사용 - 영상의 가장 먼거리 * dot 개수 차이
                penalty = ((self.clips[0].w **2 + self.clips[0].h **2) **.5) * dot_num_diff
                total_diff = penalty * dot_num_diff
                # print('total_diff', total_diff, '  penalty', penalty)

                for dot_idx in range(min_dot_num):
                    total_diff += ((ref_frame_dots[dot_idx][0] - compare_frame_dots[dot_idx][0]) **2 + \
                                   (ref_frame_dots[dot_idx][1] - compare_frame_dots[dot_idx][1]) **2) **.5
                    dist_arr.append(total_diff)
            else : 
                dist_arr.append(None)
        print(f'dist_arr : {dist_arr}')
        return dist_arr
    
    def distance(self, reference_clip, compare_clip):
        clips_frame_info = self.extract_boxes(reference_clip, compare_clip)
        min_size = min(len(clips_frame_info[0]), len(clips_frame_info[1]))
        dist_arr = self.get_all_frame_distance(clips_frame_info, min_size)
        min_diff = np.float("Inf")
        min_idx = 0
        
        for i in range(min_size - (self.MINIMAX_FRAME -1)):
            start_minimax_idx = 0 if (i -self.MINIMAX_FRAME) < 0 else i - self.MINIMAX_FRAME
            print(f'dist_arr[{start_minimax_idx} : {i + self.MINIMAX_FRAME}] : ', dist_arr[i : i + self.MINIMAX_FRAME])                
            if (None not in dist_arr[start_minimax_idx : i + self.MINIMAX_FRAME]) :
                tmp_max = np.max(dist_arr[i : i + self.MINIMAX_FRAME])
                if min_diff > tmp_max : 
                    min_diff = tmp_max
                    min_idx = i
                    print('min_idx : ', min_idx, 'tmp_max : ', tmp_max)
        
        return min_diff, (min_idx * self.SKIP_FRAME_RATE) / reference_clip.fps

In [9]:
# cross_cut.write_video(codec="libvpx")
# !cp /content/my_stagemix.mp4 /content/drive/MyDrive/Deep-Project/cross_cut
# pose_distance.model.to('cuda')

In [11]:
method = 'random'
video_path = '/content/cross_cut'
output_path = '/content/my_stagemix.mp4'
# shape_predictor_path = '/content/shape_predictor_68_face_landmarks.dat'
face_embedding_penalty = 100

print(output_path)
if method == 'random':
    random_distance = RandomDistance()
    cross_cut = Crosscut(random_distance, video_path, output_path)

elif method == 'face':
    face_distance = FaceDistance(shape_predictor_path, face_embedding_penalty)
    cross_cut = Crosscut(face_distance, video_path, output_path)

elif method == 'pose':
    pose_distance = PoseDistance()
    cross_cut = Crosscut(pose_distance, video_path, output_path)

cross_cut.generate_video()

/content/my_stagemix.mp4
LOGGER -- 6 will be mixed
current video_idx : 0 , compare video_idx : 1, distance : 15, compared end_time : 15
current video_idx : 0 , compare video_idx : 2, distance : 30, compared end_time : 15
current video_idx : 0 , compare video_idx : 3, distance : 52, compared end_time : 15
current video_idx : 0 , compare video_idx : 4, distance : 1, compared end_time : 15
current video_idx : 0 , compare video_idx : 5, distance : 81, compared end_time : 15
insert video idx : 4, insert time : 10 to 15
insert idx : 4
Not None
[MoviePy] >>>> Building video /content/my_stagemix.mp4
[MoviePy] Writing audio in my_stagemixTEMP_MPY_wvf_snd.mp3


100%|██████████| 4266/4266 [00:13<00:00, 324.92it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/my_stagemix.mp4



100%|██████████| 750/750 [02:23<00:00,  5.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/my_stagemix.mp4 



In [12]:
!cp /content/my_stagemix.mp4 /content/drive/MyDrive/Deep-Project/cross_cut/